# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [59]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

data_path = 'D:\\learning_ai\\D29\\'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [60]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df['Embarked'] = df['Embarked'].fillna('NoData')

#mean, mode, median, max
df_tmp = df.groupby(['Embarked'])['Fare'].mean().reset_index()
df_tmp.columns = ['Embarked', 'Fare_mean']
df_tmp['Fare_mode'] = df.groupby(['Embarked'])['Fare'].apply(lambda x: x.mode()).reset_index()['Fare']
df_tmp['Fare_median'] = df.groupby(['Embarked'])['Fare'].median().reset_index()['Fare']
df_tmp['Fare_max'] = df.groupby(['Embarked'])['Fare'].max().reset_index()['Fare']

df = pd.merge(df, df_tmp, on = ['Embarked'], how = 'left')

df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_mean,Fare_mode,Fare_median,Fare_max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,27.079812,8.0500,13.00,263.0000
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,59.954144,7.2292,29.70,512.3292
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,27.079812,8.0500,13.00,263.0000
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,27.079812,8.0500,13.00,263.0000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,27.079812,8.0500,13.00,263.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,27.079812,8.0500,13.00,263.0000
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,27.079812,8.0500,13.00,263.0000
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,27.079812,8.0500,13.00,263.0000
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,59.954144,7.2292,29.70,512.3292


In [61]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)

MMEncoder = MinMaxScaler()

df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max
0,3,22.0,1,0,7.2500,27.079812,8.0500,13.0,263.0000
1,1,38.0,1,0,71.2833,59.954144,7.2292,29.7,512.3292
2,3,26.0,0,0,7.9250,27.079812,8.0500,13.0,263.0000
3,1,35.0,1,0,53.1000,27.079812,8.0500,13.0,263.0000
4,3,35.0,0,0,8.0500,27.079812,8.0500,13.0,263.0000


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [62]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_minus = df.drop(['Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max'])
estimator = LogisticRegression()
print(cross_val_score(estimator, df_minus, train_Y, cv = 5).mean())


KeyError: "['Fare_mean' 'Fare_mode' 'Fare_median' 'Fare_max'] not found in axis"

In [ ]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
estimator = LogisticRegression()
print(cross_val_score(estimator, df, train_Y, cv = 5).mean())
